In [1]:

!pip install requests pandas plotly dash yfinance schedule python-dotenv
!pip install --upgrade plotly

import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [2]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import time
import os
from typing import Dict, List, Optional
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf

# Configurações globais
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Ambiente configurado com sucesso!")
print("Pipeline de Análise Financeira - Big Data")
print("=" * 50)

Ambiente configurado com sucesso!
Pipeline de Análise Financeira - Big Data


In [3]:
class FinancialDataIngestion:
    def __init__(self):
        self.data_storage = {}
        self.apis_status = {}

    def get_nasdaq_data(self, symbol: str = "^IXIC", period: str = "1mo") -> pd.DataFrame:
        # Coleta dados do Nasdaq usando Yahoo Finance
        try:
            print(f"Coletando dados do Nasdaq ({symbol})...")
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period)

            # Padronizar colunas
            data = data.reset_index()
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            data['Asset'] = 'NASDAQ'

            self.apis_status['NASDAQ'] = 'SUCCESS'
            print(f"Dados do Nasdaq coletados: {len(data)} registros")
            return data

        except Exception as e:
            print(f"Erro ao coletar dados do Nasdaq: {str(e)}")
            self.apis_status['NASDAQ'] = 'ERROR'
            return pd.DataFrame()

    def get_ibovespa_data(self, symbol: str = "^BVSP", period: str = "1mo") -> pd.DataFrame:
        # Coleta dados do Ibovespa usando Yahoo Finance
        try:
            print(f"Coletando dados do Ibovespa ({symbol})...")
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period)

            # Padronizar colunas
            data = data.reset_index()
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            data['Asset'] = 'IBOVESPA'

            self.apis_status['IBOVESPA'] = 'SUCCESS'
            print(f"Dados do Ibovespa coletados: {len(data)} registros")
            return data

        except Exception as e:
            print(f"Erro ao coletar dados do Ibovespa: {str(e)}")
            self.apis_status['IBOVESPA'] = 'ERROR'
            return pd.DataFrame()

    def get_bitcoin_data(self, symbol: str = "BTC-USD", period: str = "1mo") -> pd.DataFrame:
        # Coleta dados do Bitcoin usando Yahoo Finance
        try:
            print(f"Coletando dados do Bitcoin ({symbol})...")
            ticker = yf.Ticker(symbol)
            data = ticker.history(period=period)

            # Padronizar colunas
            data = data.reset_index()
            data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
            data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
            data['Asset'] = 'BITCOIN'

            self.apis_status['BITCOIN'] = 'SUCCESS'
            print(f"Dados do Bitcoin coletados: {len(data)} registros")
            return data

        except Exception as e:
            print(f"Erro ao coletar dados do Bitcoin: {str(e)}")
            self.apis_status['BITCOIN'] = 'ERROR'
            return pd.DataFrame()

    def collect_all_data(self) -> pd.DataFrame:
        # Coleta dados de todos os ativos
        print("=" * 50)

        all_data = []

        # Coletar dados de cada ativo
        nasdaq_data = self.get_nasdaq_data()
        if not nasdaq_data.empty:
            all_data.append(nasdaq_data)

        ibovespa_data = self.get_ibovespa_data()
        if not ibovespa_data.empty:
            all_data.append(ibovespa_data)

        bitcoin_data = self.get_bitcoin_data()
        if not bitcoin_data.empty:
            all_data.append(bitcoin_data)

        # Combinar todos os dados
        if all_data:
            combined_data = pd.concat(all_data, ignore_index=True)
            print(f"\nTotal de dados coletados: {len(combined_data)} registros")
            return combined_data
        else:
            print("Nenhum dado foi coletado com sucesso")
            return pd.DataFrame()

ingestion = FinancialDataIngestion()

In [4]:
class FinancialDataTransformation:
    # Classe responsável pela transformação e limpeza dos dados financeiros

    def __init__(self):
        self.processed_data = None

    def clean_data(self, df: pd.DataFrame) -> pd.DataFrame:

        clean_df = df.copy()


        clean_df['Date'] = pd.to_datetime(clean_df['Date'], utc=True)


        initial_rows = len(clean_df)
        clean_df = clean_df.dropna()
        removed_rows = initial_rows - len(clean_df)


        clean_df = clean_df.sort_values(['Asset', 'Date']).reset_index(drop=True)

        print(f"Limpeza concluída:")
        print(f"   - Linhas removidas (NaN): {removed_rows}")
        print(f"   - Linhas finais: {len(clean_df)}")

        return clean_df

    def calculate_returns(self, df: pd.DataFrame) -> pd.DataFrame:
        # Calcula retornos diários, semanais e mensais


        df_returns = df.copy()

        # Calcular retornos para cada ativo
        for asset in df_returns['Asset'].unique():
            mask = df_returns['Asset'] == asset
            asset_data = df_returns[mask].copy()

            # Retorno diário
            asset_data['Daily_Return'] = asset_data['Close'].pct_change()

            # Retorno acumulado
            asset_data['Cumulative_Return'] = (1 + asset_data['Daily_Return']).cumprod() - 1

            # Volatilidade móvel (7 dias)
            asset_data['Volatility_7d'] = asset_data['Daily_Return'].rolling(window=7).std()

            # Média móvel (7 dias)
            asset_data['MA_7'] = asset_data['Close'].rolling(window=7).mean()

            # Atualizar dados no DataFrame principal
            df_returns.loc[mask, 'Daily_Return'] = asset_data['Daily_Return']
            df_returns.loc[mask, 'Cumulative_Return'] = asset_data['Cumulative_Return']
            df_returns.loc[mask, 'Volatility_7d'] = asset_data['Volatility_7d']
            df_returns.loc[mask, 'MA_7'] = asset_data['MA_7']

        print("Cálculo de retornos concluído")
        return df_returns

    def aggregate_by_period(self, df: pd.DataFrame) -> pd.DataFrame:
        # Agrega dados por períodos (semanal, mensal)
        print("Agregando dados por período...")

        # Adicionar colunas de período
        df['Year_Month'] = df['Date'].dt.to_period('M')
        df['Year_Week'] = df['Date'].dt.to_period('W')

        # Agregação mensal
        monthly_agg = df.groupby(['Asset', 'Year_Month']).agg({
            'Open': 'first',
            'Close': 'last',
            'High': 'max',
            'Low': 'min',
            'Volume': 'sum',
            'Daily_Return': 'sum'  # Retorno mensal
        }).reset_index()

        monthly_agg['Period_Type'] = 'Monthly'
        monthly_agg['Period'] = monthly_agg['Year_Month'].astype(str)

        print("Agregação concluída")
        return monthly_agg


    def process_pipeline(self, raw_data: pd.DataFrame) -> pd.DataFrame:
        # Executa todo o pipeline de transformação
        print("INICIANDO PIPELINE DE TRANSFORMAÇÃO")
        print("=" * 50)

        # 1. Limpeza
        clean_data = self.clean_data(raw_data)

        # 2. Cálculo de retornos
        data_with_returns = self.calculate_returns(clean_data)

        # 3. Agregação
        monthly_data = self.aggregate_by_period(data_with_returns)

        # Armazenar dados processados
        self.processed_data = data_with_returns

        print("\nPipeline de transformação concluído!")
        return data_with_returns

# Criar instância da classe
transformation = FinancialDataTransformation()

In [5]:
class FinancialDataStorage:

    # Classe responsável pelo armazenamento dos dados processados


    def __init__(self):
        self.storage_path = "/content/financial_data/"
        self.create_storage_structure()

    def create_storage_structure(self):
        import os

        folders = [
            self.storage_path,
            f"{self.storage_path}raw/",
            f"{self.storage_path}processed/",
            f"{self.storage_path}aggregated/"
        ]

        for folder in folders:
            os.makedirs(folder, exist_ok=True)

        print("Estrutura de armazenamento criada")

    def save_raw_data(self, data: pd.DataFrame, filename: str = None) -> str:
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"raw_data_{timestamp}.csv"

        filepath = f"{self.storage_path}raw/{filename}"
        data.to_csv(filepath, index=False)

        print(f"Dados brutos salvos: {filepath}")
        return filepath

    def save_processed_data(self, data: pd.DataFrame, filename: str = None) -> str:
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"processed_data_{timestamp}.csv"

        filepath = f"{self.storage_path}processed/{filename}"
        data.to_csv(filepath, index=False)

        print(f"Dados processados salvos: {filepath}")
        return filepath

    def save_to_parquet(self, data: pd.DataFrame, filename: str = None) -> str:
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"financial_data_{timestamp}.parquet"

        filepath = f"{self.storage_path}processed/{filename}"
        data.to_parquet(filepath, index=False)

        print(f"Dados salvos em Parquet (Data Lake): {filepath}")
        return filepath

    def get_storage_summary(self) -> Dict:
        import os

        summary = {
            "raw_files": len(os.listdir(f"{self.storage_path}raw/")) if os.path.exists(f"{self.storage_path}raw/") else 0,
            "processed_files": len(os.listdir(f"{self.storage_path}processed/")) if os.path.exists(f"{self.storage_path}processed/") else 0,
            "total_size_mb": 0
        }

        for root, dirs, files in os.walk(self.storage_path):
            for file in files:
                file_path = os.path.join(root, file)
                summary["total_size_mb"] += os.path.getsize(file_path)

        summary["total_size_mb"] = round(summary["total_size_mb"] / (1024 * 1024), 2)

        return summary

storage = FinancialDataStorage()

Estrutura de armazenamento criada


In [7]:
raw_data = ingestion.collect_all_data()


if not raw_data.empty:
    print(f"\nResumo dos dados coletados:")
    print(f"Total de registros: {len(raw_data)}")
    print(f"Período: {raw_data['Date'].min().date()} a {raw_data['Date'].max().date()}")
    print(f"Ativos: {raw_data['Asset'].unique().tolist()}")

    print(f"\nPrimeiras linhas dos dados:")
    display(raw_data.head())


else:
    print("Nenhum dado foi coletado.")

Coletando dados do Nasdaq (^IXIC)...
Dados do Nasdaq coletados: 22 registros
Coletando dados do Ibovespa (^BVSP)...
Dados do Ibovespa coletados: 23 registros
Coletando dados do Bitcoin (BTC-USD)...
Dados do Bitcoin coletados: 31 registros

Total de dados coletados: 76 registros

Resumo dos dados coletados:
Total de registros: 76
Período: 2025-10-28 a 2025-12-01
Ativos: ['NASDAQ', 'IBOVESPA', 'BITCOIN']

Primeiras linhas dos dados:


,Date,Open,High,Low,Close,Volume,Asset
0,2025-10-29 00:00:00-04:00,23987.289062,24019.990234,23763.990234,23958.470703,10089540000,NASDAQ
1,2025-10-30 00:00:00-04:00,23793.080078,23846.130859,23578.470703,23581.140625,10564160000,NASDAQ
2,2025-10-31 00:00:00-04:00,23941.779297,23946.230469,23628.820312,23724.960938,10909250000,NASDAQ
3,2025-11-03 00:00:00-05:00,23951.910156,23976.839844,23764.869141,23834.720703,10360170000,NASDAQ
4,2025-11-04 00:00:00-05:00,23458.210938,23644.150391,23333.320312,23348.640625,10227700000,NASDAQ


In [8]:
# Executar transformação dos dados
if not raw_data.empty:
    processed_data = transformation.process_pipeline(raw_data)

    # Mostrar resultados
    print(f"\nResumo dos dados processados:")
    print(f"Total de registros: {len(processed_data)}")
    print(f"Colunas adicionadas: Daily_Return, Cumulative_Return, Volatility_7d, MA_7")

    # Estatísticas por ativo
    print(f"\nEstatísticas por ativo:")
    for asset in processed_data['Asset'].unique():
        asset_data = processed_data[processed_data['Asset'] == asset]
        avg_return = asset_data['Daily_Return'].mean() * 100
        volatility = asset_data['Daily_Return'].std() * 100

        print(f"{asset}:")
        print(f"   - Retorno médio diário: {avg_return:.3f}%")
        print(f"   - Volatilidade: {volatility:.3f}%")
        print(f"   - Registros: {len(asset_data)}")

    # Mostrar dados processados
    print(f"\nAmostra dos dados processados:")
    display(processed_data[['Date', 'Asset', 'Close', 'Daily_Return', 'Cumulative_Return']].head(10))
else:
    print("Não há dados para processar. Execute primeiro a ingestão.")

INICIANDO PIPELINE DE TRANSFORMAÇÃO
Limpeza concluída:
   - Linhas removidas (NaN): 0
   - Linhas finais: 76
Cálculo de retornos concluído
Agregando dados por período...
Agregação concluída

Pipeline de transformação concluído!

Resumo dos dados processados:
Total de registros: 76
Colunas adicionadas: Daily_Return, Cumulative_Return, Volatility_7d, MA_7

Estatísticas por ativo:
BITCOIN:
   - Retorno médio diário: -0.778%
   - Volatilidade: 2.431%
   - Registros: 31
IBOVESPA:
   - Retorno médio diário: 0.348%
   - Volatilidade: 0.679%
   - Registros: 23
NASDAQ:
   - Retorno médio diário: -0.110%
   - Volatilidade: 1.384%
   - Registros: 22

Amostra dos dados processados:


,Date,Asset,Close,Daily_Return,Cumulative_Return
0,2025-11-01 00:00:00+00:00,BITCOIN,110064.015625,NaN,NaN
1,2025-11-02 00:00:00+00:00,BITCOIN,110639.625000,0.005230,0.005230
2,2025-11-03 00:00:00+00:00,BITCOIN,106547.523438,-0.036986,-0.031950
3,2025-11-04 00:00:00+00:00,BITCOIN,101590.523438,-0.046524,-0.076987
4,2025-11-05 00:00:00+00:00,BITCOIN,103891.835938,0.022653,-0.056078
5,2025-11-06 00:00:00+00:00,BITCOIN,101301.289062,-0.024935,-0.079615
6,2025-11-07 00:00:00+00:00,BITCOIN,103372.406250,0.020445,-0.060797
7,2025-11-08 00:00:00+00:00,BITCOIN,102282.117188,-0.010547,-0.070703
8,2025-11-09 00:00:00+00:00,BITCOIN,104719.640625,0.023831,-0.048557
9,2025-11-10 00:00:00+00:00,BITCOIN,105996.593750,0.012194,-0.036955


In [9]:
# Executar armazenamento dos dados
if 'processed_data' in locals() and not processed_data.empty:

    raw_file = storage.save_raw_data(raw_data, "financial_raw_data.csv")

    processed_file = storage.save_processed_data(processed_data, "financial_processed_data.csv")

    parquet_file = storage.save_to_parquet(processed_data, "financial_data_lake.parquet")

    summary = storage.get_storage_summary()
    print(f"\nResumo do Armazenamento:")
    print(f"   - Arquivos raw: {summary['raw_files']}")
    print(f"   - Arquivos processados: {summary['processed_files']}")
    print(f"   - Tamanho total: {summary['total_size_mb']} MB")

    print(f"\nDados armazenados com sucesso!")
else:
    print("Não há dados processados para armazenar.")

Dados brutos salvos: /content/financial_data/raw/financial_raw_data.csv
Dados processados salvos: /content/financial_data/processed/financial_processed_data.csv
Dados salvos em Parquet (Data Lake): /content/financial_data/processed/financial_data_lake.parquet

Resumo do Armazenamento:
   - Arquivos raw: 1
   - Arquivos processados: 2
   - Tamanho total: 0.03 MB

Dados armazenados com sucesso!


In [26]:
class FinancialDashboard:

    def __init__(self, data: pd.DataFrame):
        self.data = data

    def plot_price_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset]

            fig.add_trace(go.Scatter(
                x=asset_data['Date'],
                y=asset_data['Close'],
                mode='lines',
                name=asset,
                line=dict(width=2)
            ))

        fig.update_layout(
            title="Comparação de Preços dos Ativos",
            xaxis_title="Data",
            yaxis_title="Preço de Fechamento",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_returns_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset]

            fig.add_trace(go.Scatter(
                x=asset_data['Date'],
                y=asset_data['Cumulative_Return'] * 100,
                mode='lines',
                name=f"{asset} - Retorno Acumulado",
                line=dict(width=2)
            ))

        fig.update_layout(
            title="Comparação de Retornos Acumulados",
            xaxis_title="Data",
            yaxis_title="Retorno Acumulado (%)",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_volatility_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset].dropna(subset=['Volatility_7d'])

            if not asset_data.empty:
                fig.add_trace(go.Scatter(
                    x=asset_data['Date'],
                    y=asset_data['Volatility_7d'] * 100,
                    mode='lines',
                    name=f"{asset} - Volatilidade 7d",
                    line=dict(width=2)
                ))

        fig.update_layout(
            title="Comparação de Volatilidade (7 dias)",
            xaxis_title="Data",
            yaxis_title="Volatilidade (%)",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_volume_comparison(self) -> go.Figure:
        fig = make_subplots(
            rows=3, cols=1,
            subplot_titles=tuple(f"Volume - {asset}" for asset in self.data['Asset'].unique()),
            vertical_spacing=0.1
        )

        for i, asset in enumerate(self.data['Asset'].unique(), 1):
            asset_data = self.data[self.data['Asset'] == asset]

            fig.add_trace(
                go.Bar(
                    x=asset_data['Date'],
                    y=asset_data['Volume'],
                    name=f"{asset} Volume",
                    showlegend=False
                ),
                row=i, col=1
            )

        fig.update_layout(
            title="Comparação de Volume de Negociação",
            template="plotly_white",
            height=800
        )

        return fig

    def plot_moving_average_comparison(self) -> go.Figure:
        fig = go.Figure()

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset].dropna(subset=['MA_7'])

            if not asset_data.empty:
                fig.add_trace(go.Scatter(
                    x=asset_data['Date'],
                    y=asset_data['MA_7'],
                    mode='lines',
                    name=f"{asset} - MA 7d",
                    line=dict(width=2)
                ))

        fig.update_layout(
            title="Comparação de Médias Móveis (7 dias)",
            xaxis_title="Data",
            yaxis_title="Média Móvel de Fechamento (7 dias)",
            template="plotly_white",
            height=500
        )

        return fig

    def plot_candlestick_chart(self, asset_name: str) -> go.Figure:
        asset_data = self.data[self.data['Asset'] == asset_name]

        if asset_data.empty:
            print(f"Dados não encontrados para o ativo: {asset_name}")
            return go.Figure()

        fig = go.Figure(data=[go.Candlestick(
            x=asset_data['Date'],
            open=asset_data['Open'],
            high=asset_data['High'],
            low=asset_data['Low'],
            close=asset_data['Close']
        )])

        fig.update_layout(
            title=f"Gráfico de Candlestick - {asset_name}",
            xaxis_title="Data",
            yaxis_title="Preço",
            template="plotly_white",
            height=500
        )

        return fig


    def create_summary_table(self) -> pd.DataFrame:
        summary_data = []

        for asset in self.data['Asset'].unique():
            asset_data = self.data[self.data['Asset'] == asset]

            summary_data.append({
                'Ativo': asset,
                'Período': f"{asset_data['Date'].min().date()} a {asset_data['Date'].max().date()}",
                'Preço Inicial': f"${asset_data['Close'].iloc[0]:,.2f}",
                'Preço Final': f"${asset_data['Close'].iloc[-1]:,.2f}",
                'Retorno Total (%)': f"{(asset_data['Cumulative_Return'].iloc[-1] * 100):.2f}%",
                'Retorno Médio Diário (%)': f"{(asset_data['Daily_Return'].mean() * 100):.3f}%",
                'Volatilidade (%)': f"{(asset_data['Daily_Return'].std() * 100):.3f}%",
                'Volume Médio': f"{asset_data['Volume'].mean():,.0f}"
            })

        return pd.DataFrame(summary_data)

    def plot_correlation_heatmap(self) -> go.Figure:
        """Matriz de correlação entre os retornos dos ativos"""
        # Pivot dos retornos diários por ativo
        returns_pivot = self.data.pivot(
            index='Date',
            columns='Asset',
            values='Daily_Return'
        )

        # Calcular matriz de correlação
        corr_matrix = returns_pivot.corr()

        fig = go.Figure(data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.columns,
            colorscale='RdBu',
            zmid=0,
            text=np.round(corr_matrix.values, 2),
            texttemplate='%{text}',
            textfont={"size": 12},
            colorbar=dict(title="Correlação")
        ))

        fig.update_layout(
            title="Matriz de Correlação entre Retornos dos Ativos",
            template="plotly_white",
            height=500,
            width=600
        )

        return fig

    def plot_risk_return_scatter(self) -> go.Figure:
      """Gráfico de risco vs retorno (scatter plot)"""
      summary_data = []

      for asset in self.data['Asset'].unique():
          asset_data = self.data[self.data['Asset'] == asset]

          # Calcular métricas
          avg_return = asset_data['Daily_Return'].mean() * 252 * 100  # Anualizado
          volatility = asset_data['Daily_Return'].std() * np.sqrt(252) * 100  # Anualizado
          sharpe_ratio = avg_return / volatility if volatility != 0 else 0

          summary_data.append({
              'Asset': asset,
              'Retorno Anual (%)': avg_return,
              'Volatilidade Anual (%)': volatility,
              'Sharpe Ratio': sharpe_ratio
          })

      df_summary = pd.DataFrame(summary_data)

      fig = go.Figure()

      fig.add_trace(go.Scatter(
          x=df_summary['Volatilidade Anual (%)'],
          y=df_summary['Retorno Anual (%)'],
          mode='markers+text',
          marker=dict(size=15, color=df_summary['Sharpe Ratio'],
                      colorscale='Viridis', showscale=True,
                      colorbar=dict(title="Sharpe Ratio")),
          text=df_summary['Asset'],
          textposition="top center",
          textfont=dict(size=12),
          hovertemplate='<b>%{text}</b><br>' +
                        'Retorno: %{y:.2f}%<br>' +
                        'Volatilidade: %{x:.2f}%<br>' +
                        '<extra></extra>'
      ))

      fig.update_layout(
          title="Risco vs Retorno (Anualizado)",
          xaxis_title="Volatilidade Anual (%)",
          yaxis_title="Retorno Anual (%)",
          template="plotly_white",
          height=500
      )

      return fig

    def plot_daily_returns_distribution(self) -> go.Figure:
      """Distribuição dos retornos diários (histograma)"""
      fig = go.Figure()

      for asset in self.data['Asset'].unique():
          asset_data = self.data[self.data['Asset'] == asset]

          fig.add_trace(go.Histogram(
              x=asset_data['Daily_Return'] * 100,
              name=asset,
              opacity=0.7,
              nbinsx=30
          ))

      fig.update_layout(
          title="Distribuição dos Retornos Diários",
          xaxis_title="Retorno Diário (%)",
          yaxis_title="Frequência",
          barmode='overlay',
          template="plotly_white",
          height=500
      )

      return fig

    def plot_drawdown(self) -> go.Figure:
      """Gráfico de drawdown (queda máxima do pico)"""
      fig = go.Figure()

      for asset in self.data['Asset'].unique():
          asset_data = self.data[self.data['Asset'] == asset].copy()

          # Calcular drawdown
          cumulative = (1 + asset_data['Daily_Return']).cumprod()
          running_max = cumulative.expanding().max()
          drawdown = (cumulative - running_max) / running_max * 100

          fig.add_trace(go.Scatter(
              x=asset_data['Date'],
              y=drawdown,
              mode='lines',
              name=asset,
              fill='tozeroy',
              line=dict(width=2)
          ))

      fig.update_layout(
          title="Drawdown dos Ativos (% de queda do pico)",
          xaxis_title="Data",
          yaxis_title="Drawdown (%)",
          template="plotly_white",
          height=500
      )

      return fig

    def plot_price_normalized(self) -> go.Figure:
      """Preços normalizados (base 100) para comparação"""
      fig = go.Figure()

      for asset in self.data['Asset'].unique():
          asset_data = self.data[self.data['Asset'] == asset].copy()

          # Normalizar para base 100
          normalized_price = (asset_data['Close'] / asset_data['Close'].iloc[0]) * 100

          fig.add_trace(go.Scatter(
              x=asset_data['Date'],
              y=normalized_price,
              mode='lines',
              name=asset,
              line=dict(width=2)
          ))

      fig.update_layout(
          title="Preços Normalizados (Base 100)",
          xaxis_title="Data",
          yaxis_title="Índice (Base 100)",
          template="plotly_white",
          height=500,
          hovermode='x unified'
      )

      return fig

    def plot_volume_price_relationship(self) -> go.Figure:
      """Relação entre volume e variação de preço"""
      fig = make_subplots(
          rows=len(self.data['Asset'].unique()),
          cols=1,
          subplot_titles=tuple(self.data['Asset'].unique()),
          vertical_spacing=0.08
      )

      for i, asset in enumerate(self.data['Asset'].unique(), 1):
          asset_data = self.data[self.data['Asset'] == asset].copy()

          # Calcular variação de preço
          asset_data['Price_Change'] = asset_data['Daily_Return'] * 100

          # Normalizar volume para melhor visualização
          volume_normalized = (asset_data['Volume'] - asset_data['Volume'].min()) / \
                              (asset_data['Volume'].max() - asset_data['Volume'].min())

          fig.add_trace(
              go.Scatter(
                  x=volume_normalized,
                  y=asset_data['Price_Change'],
                  mode='markers',
                  marker=dict(
                      size=8,
                      color=asset_data['Price_Change'],
                      colorscale='RdYlGn',
                      showscale=(i==1),
                      colorbar=dict(title="Variação (%)") if i==1 else None
                  ),
                  showlegend=False,
                  hovertemplate='Volume: %{x:.2f}<br>Variação: %{y:.2f}%<extra></extra>'
              ),
              row=i, col=1
          )

          fig.update_xaxes(title_text="Volume (normalizado)", row=i, col=1)
          fig.update_yaxes(title_text="Variação (%)", row=i, col=1)

      fig.update_layout(
          title="Relação Volume vs Variação de Preço",
          template="plotly_white",
          height=300 * len(self.data['Asset'].unique()),
          showlegend=False
      )

      return fig

    def plot_bollinger_bands(self, asset_name: str) -> go.Figure:
      """Bandas de Bollinger para um ativo específico"""
      asset_data = self.data[self.data['Asset'] == asset_name].copy()

      if asset_data.empty:
          print(f"Dados não encontrados para o ativo: {asset_name}")
          return go.Figure()

      # Calcular Bandas de Bollinger
      window = 20
      asset_data['BB_Middle'] = asset_data['Close'].rolling(window=window).mean()
      asset_data['BB_Std'] = asset_data['Close'].rolling(window=window).std()
      asset_data['BB_Upper'] = asset_data['BB_Middle'] + (asset_data['BB_Std'] * 2)
      asset_data['BB_Lower'] = asset_data['BB_Middle'] - (asset_data['BB_Std'] * 2)

      fig = go.Figure()

      # Banda superior
      fig.add_trace(go.Scatter(
          x=asset_data['Date'],
          y=asset_data['BB_Upper'],
          name='Banda Superior',
          line=dict(color='rgba(255,0,0,0.3)', width=1),
          hovertemplate='%{y:.2f}<extra></extra>'
      ))

      # Banda inferior
      fig.add_trace(go.Scatter(
          x=asset_data['Date'],
          y=asset_data['BB_Lower'],
          name='Banda Inferior',
          line=dict(color='rgba(0,255,0,0.3)', width=1),
          fill='tonexty',
          fillcolor='rgba(128,128,128,0.1)',
          hovertemplate='%{y:.2f}<extra></extra>'
      ))

      # Média móvel
      fig.add_trace(go.Scatter(
          x=asset_data['Date'],
          y=asset_data['BB_Middle'],
          name='Média Móvel (20d)',
          line=dict(color='blue', width=2),
          hovertemplate='%{y:.2f}<extra></extra>'
      ))

      # Preço de fechamento
      fig.add_trace(go.Scatter(
          x=asset_data['Date'],
          y=asset_data['Close'],
          name='Preço de Fechamento',
          line=dict(color='black', width=2),
          hovertemplate='%{y:.2f}<extra></extra>'
      ))

      fig.update_layout(
          title=f"Bandas de Bollinger - {asset_name}",
          xaxis_title="Data",
          yaxis_title="Preço",
          template="plotly_white",
          height=500,
          hovermode='x unified'
      )

      return fig


# Criar instância da classe
if 'processed_data' in locals() and not processed_data.empty:
    dashboard = FinancialDashboard(processed_data)
else:
    print("Não há dados processados para criar o dashboard.")

In [30]:
# Gerar todas as visualizações
if 'dashboard' in locals():
    # # 1. Tabela Resumo
    # print("Tabela Resumo dos Ativos:")
    # summary_table = dashboard.create_summary_table()
    # display(summary_table)

    # # 2. Gráfico de Preços
    # print("\nGerando gráfico de comparação de preços...")
    # price_fig = dashboard.plot_price_comparison()
    # price_fig.show()

    # 3. Gráfico de Retornos
    print("\nGerando gráfico de retornos acumulados...")
    returns_fig = dashboard.plot_returns_comparison()
    returns_fig.show()

    # 4. Gráfico de Volatilidade
    print("\nGerando gráfico de volatilidade...")
    volatility_fig = dashboard.plot_volatility_comparison()
    volatility_fig.show()

    # 5. Gráfico de Volume
    print("\nGerando gráfico de volume...")
    volume_fig = dashboard.plot_volume_comparison()
    volume_fig.show()

    # # 6. Gráfico de Médias Móveis
    # print("\nGerando gráfico de médias móveis...")
    # ma_fig = dashboard.plot_moving_average_comparison()
    # ma_fig.show()

    # # 7. Matriz de Correlação
    # print("\nGerando matriz de correlação...")
    # corr_fig = dashboard.plot_correlation_heatmap()
    # corr_fig.show()

    # 8. Risco vs Retorno
    print("\nGerando gráfico de risco vs retorno...")
    risk_return_fig = dashboard.plot_risk_return_scatter()
    risk_return_fig.show()

    # # 9. Distribuição de Retornos
    # print("\nGerando distribuição de retornos diários...")
    # dist_fig = dashboard.plot_daily_returns_distribution()
    # dist_fig.show()

    # 10. Drawdown
    print("\nGerando gráfico de drawdown...")
    drawdown_fig = dashboard.plot_drawdown()
    drawdown_fig.show()

    # # 11. Preços Normalizados
    # print("\nGerando preços normalizados...")
    # normalized_fig = dashboard.plot_price_normalized()
    # normalized_fig.show()

    # 12. Volume vs Variação de Preço
    print("\nGerando relação volume vs variação de preço...")
    vol_price_fig = dashboard.plot_volume_price_relationship()
    vol_price_fig.show()

    # 13. Bandas de Bollinger (para cada ativo)
    print("\nGerando Bandas de Bollinger...")
    for asset in processed_data['Asset'].unique():
        bb_fig = dashboard.plot_bollinger_bands(asset)
        bb_fig.show()

    print("\nTodas as visualizações foram geradas com sucesso")
else:
    print("Dashboard não está disponível. Execute primeiro as células anteriores.")


Gerando gráfico de retornos acumulados...



Gerando gráfico de volatilidade...



Gerando gráfico de volume...



Gerando gráfico de risco vs retorno...



Gerando gráfico de drawdown...



Gerando relação volume vs variação de preço...



Gerando Bandas de Bollinger...



Todas as visualizações foram geradas com sucesso


In [12]:
if 'processed_data' in locals() and not processed_data.empty:

    print(f"\nRESULTADOS OBTIDOS:")
    print(f"• Ativos analisados: {', '.join(processed_data['Asset'].unique())}")
    print(f"• Período: {processed_data['Date'].min().date()} a {processed_data['Date'].max().date()}")
    print(f"• Total de dados: {len(processed_data)} registros")

    # Insights rápidos
    print(f"\nINSIGHTS PRELIMINARES:")
    for asset in processed_data['Asset'].unique():
        asset_data = processed_data[processed_data['Asset'] == asset]
        total_return = asset_data['Cumulative_Return'].iloc[-1] * 100
        volatility = asset_data['Daily_Return'].std() * 100
        print(f"• {asset}: Retorno {total_return:.2f}%, Volatilidade {volatility:.2f}%")
else:
    print("Erro: Pipeline não está funcionando. Verifique as células anteriores.")


RESULTADOS OBTIDOS:
• Ativos analisados: BITCOIN, IBOVESPA, NASDAQ
• Período: 2025-10-28 a 2025-12-01
• Total de dados: 76 registros

INSIGHTS PRELIMINARES:
• BITCOIN: Retorno -21.58%, Volatilidade 2.43%
• IBOVESPA: Retorno 7.90%, Volatilidade 0.68%
• NASDAQ: Retorno -2.47%, Volatilidade 1.38%


## Preparar Dados para o Modelo de ML




In [13]:
from sklearn.preprocessing import MinMaxScaler

asset_data_for_ml = {}

# Define feature and target columns
X_cols = ['Open', 'High', 'Low', 'Volume', 'Daily_Return', 'Volatility_7d', 'MA_7']
y_col = 'Close'

# Iterate through each unique asset
for asset in processed_data['Asset'].unique():
    print(f"Preparando dados para o ativo: {asset}")

    # a. Filter data for the current asset
    asset_df = processed_data[processed_data['Asset'] == asset].copy()

    # b. and c. Define feature and target DataFrames/Series
    X = asset_df[X_cols]
    y = asset_df[y_col]

    # e. Drop NaN values and reset index to ensure alignment
    # Combine X and y to drop rows consistently
    combined_df = pd.concat([X, y], axis=1).dropna()
    X = combined_df[X_cols]
    y = combined_df[y_col]

    if X.empty or y.empty:
        print(f"    Nenhum dado válido após remoção de NaNs para {asset}. Pulando.")
        continue

    # Reset index after dropping NaNs
    X = X.reset_index(drop=True)
    y = y.reset_index(drop=True)

    # f. Initialize separate MinMaxScaler objects
    feature_scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()

    # g. Fit and transform features
    X_scaled = feature_scaler.fit_transform(X)
    X_scaled_df = pd.DataFrame(X_scaled, columns=X_cols)

    # h. Fit and transform target
    y_scaled = target_scaler.fit_transform(y.values.reshape(-1, 1))
    y_scaled_series = pd.Series(y_scaled.flatten())

    # i. Determine the split point (80% for training, 20% for testing)
    split_point = int(len(X_scaled_df) * 0.8)

    # j. Split the scaled X and y into train and test sets
    X_train = X_scaled_df.iloc[:split_point]
    X_test = X_scaled_df.iloc[split_point:]

    y_train = y_scaled_series.iloc[:split_point]
    y_test = y_scaled_series.iloc[split_point:]

    # k. Store all prepared data and scalers in the dictionary
    asset_data_for_ml[asset] = {
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'feature_scaler': feature_scaler,
        'target_scaler': target_scaler
    }
    print(f"    Dados para {asset} preparados: Treino ({len(X_train)}), Teste ({len(X_test)}) registros.")

print("\nPreparação dos dados para ML concluída para todos os ativos.")

Preparando dados para o ativo: BITCOIN
    Dados para BITCOIN preparados: Treino (19), Teste (5) registros.
Preparando dados para o ativo: IBOVESPA
    Dados para IBOVESPA preparados: Treino (12), Teste (4) registros.
Preparando dados para o ativo: NASDAQ
    Dados para NASDAQ preparados: Treino (12), Teste (3) registros.

Preparação dos dados para ML concluída para todos os ativos.


In [14]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

asset_data_for_ml = {}

# Define feature and target columns
X_cols = ['Open', 'High', 'Low', 'Volume', 'Daily_Return', 'Volatility_7d', 'MA_7']
y_col = 'Close'

# Check if processed_data is defined and not empty
if 'processed_data' in locals() and not processed_data.empty:
    # Iterate through each unique asset
    for asset in processed_data['Asset'].unique():
        print(f"Preparando dados para o ativo: {asset}")

        # a. Filter data for the current asset
        asset_df = processed_data[processed_data['Asset'] == asset].copy()

        # b. and c. Define feature and target DataFrames/Series
        X = asset_df[X_cols]
        y = asset_df[y_col]

        # e. Drop NaN values and reset index to ensure alignment
        # Combine X and y to drop rows consistently
        combined_df = pd.concat([X, y], axis=1).dropna()
        X = combined_df[X_cols]
        y = combined_df[y_col]

        if X.empty or y.empty:
            print(f"    Nenhum dado válido após remoção de NaNs para {asset}. Pulando.")
            continue

        # Reset index after dropping NaNs
        X = X.reset_index(drop=True)
        y = y.reset_index(drop=True)

        # f. Initialize separate MinMaxScaler objects
        feature_scaler = MinMaxScaler()
        target_scaler = MinMaxScaler()

        # g. Fit and transform features
        X_scaled = feature_scaler.fit_transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=X_cols)

        # h. Fit and transform target
        y_scaled = target_scaler.fit_transform(y.values.reshape(-1, 1))
        y_scaled_series = pd.Series(y_scaled.flatten())

        # i. Determine the split point (80% for training, 20% for testing)
        split_point = int(len(X_scaled_df) * 0.8)

        # j. Split the scaled X and y into train and test sets
        X_train = X_scaled_df.iloc[:split_point]
        X_test = X_scaled_df.iloc[split_point:]

        y_train = y_scaled_series.iloc[:split_point]
        y_test = y_scaled_series.iloc[split_point:]

        # k. Store all prepared data and scalers in the dictionary
        asset_data_for_ml[asset] = {
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test,
            'feature_scaler': feature_scaler,
            'target_scaler': target_scaler
        }
        print(f"    Dados para {asset} preparados: Treino ({len(X_train)}), Teste ({len(X_test)}) registros.")

    print("\nPreparação dos dados para ML concluída para todos os ativos.")
else:
    print("Erro: 'processed_data' não está disponível ou está vazio. Por favor, execute as etapas de ingestão e transformação de dados primeiro.")

Preparando dados para o ativo: BITCOIN
    Dados para BITCOIN preparados: Treino (19), Teste (5) registros.
Preparando dados para o ativo: IBOVESPA
    Dados para IBOVESPA preparados: Treino (12), Teste (4) registros.
Preparando dados para o ativo: NASDAQ
    Dados para NASDAQ preparados: Treino (12), Teste (3) registros.

Preparação dos dados para ML concluída para todos os ativos.


In [15]:
print("Dados preparados para o modelo de ML com sucesso!")

Dados preparados para o modelo de ML com sucesso!


## Definir e Treinar o Modelo de ML



In [16]:
from sklearn.linear_model import LinearRegression

trained_models = {}


for asset, data in asset_data_for_ml.items():
    print(f"Treinando modelo de Regressão Linear para {asset}...")

    X_train = data['X_train']
    y_train = data['y_train']


    model = LinearRegression()


    model.fit(X_train, y_train)


    trained_models[asset] = model
    print(f"Modelo para {asset} treinado com sucesso.")

print("\nTreinamento de modelos de Machine Learning concluído para todos os ativos.")

Treinando modelo de Regressão Linear para BITCOIN...
Modelo para BITCOIN treinado com sucesso.
Treinando modelo de Regressão Linear para IBOVESPA...
Modelo para IBOVESPA treinado com sucesso.
Treinando modelo de Regressão Linear para NASDAQ...
Modelo para NASDAQ treinado com sucesso.

Treinamento de modelos de Machine Learning concluído para todos os ativos.


## Avaliar o Modelo de ML




In [17]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


evaluation_results = {}


for asset, model in trained_models.items():
    print(f"Avaliando modelo para o ativo: {asset}...")

    data = asset_data_for_ml[asset]
    X_test = data['X_test']
    y_test = data['y_test']
    target_scaler = data['target_scaler']

    y_pred_scaled = model.predict(X_test)

    y_test_unscaled = target_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten()
    y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

    mse = mean_squared_error(y_test_unscaled, y_pred_unscaled)

    rmse = np.sqrt(mse)

    r2 = r2_score(y_test_unscaled, y_pred_unscaled)

    evaluation_results[asset] = {
        'MSE': mse,
        'RMSE': rmse,
        'R-squared': r2
    }

    print(f"  Avaliação concluída para {asset}.")

print("\nResultados da Avaliação do Modelo:")
print("=" * 30)

for asset, metrics in evaluation_results.items():
    print(f"Ativo: {asset}")
    print(f"  MSE: {metrics['MSE']:.4f}")
    print(f"  RMSE: {metrics['RMSE']:.4f}")
    print(f"  R-squared: {metrics['R-squared']:.4f}")
    print("-" * 20)

print("Avaliação de modelos de Machine Learning concluída para todos os ativos.")

Avaliando modelo para o ativo: BITCOIN...
  Avaliação concluída para BITCOIN.
Avaliando modelo para o ativo: IBOVESPA...
  Avaliação concluída para IBOVESPA.
Avaliando modelo para o ativo: NASDAQ...
  Avaliação concluída para NASDAQ.

Resultados da Avaliação do Modelo:
Ativo: BITCOIN
  MSE: 39046.0432
  RMSE: 197.6007
  R-squared: 0.9885
--------------------
Ativo: IBOVESPA
  MSE: 665.5898
  RMSE: 25.7990
  R-squared: 0.9995
--------------------
Ativo: NASDAQ
  MSE: 52753.0365
  RMSE: 229.6803
  R-squared: -1.7250
--------------------
Avaliação de modelos de Machine Learning concluída para todos os ativos.


In [18]:
predictions_df = {}

for asset, model in trained_models.items():
    print(f"Gerando previsões para o ativo: {asset}...")

    data = asset_data_for_ml[asset]
    X_test = data['X_test']
    y_test = data['y_test']
    target_scaler = data['target_scaler']

    y_pred_scaled = model.predict(X_test)

    y_test_unscaled = target_scaler.inverse_transform(y_test.values.reshape(-1, 1)).flatten()
    y_pred_unscaled = target_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()


    asset_original_df = processed_data[processed_data['Asset'] == asset].copy()
    combined_original_df = pd.concat([asset_original_df[X_cols], asset_original_df[y_col]], axis=1).dropna()
    test_dates = asset_original_df.loc[combined_original_df.index[len(data['X_train']):]]['Date'].reset_index(drop=True)


    predictions_df[asset] = {
        'dates': test_dates,
        'actual': y_test_unscaled,
        'predicted': y_pred_unscaled
    }
    print(f"  Previsões para {asset} geradas com sucesso. ({len(y_pred_unscaled)} registros)")

print("\nGeração de previsões concluída para todos os ativos.")

Gerando previsões para o ativo: BITCOIN...
  Previsões para BITCOIN geradas com sucesso. (5 registros)
Gerando previsões para o ativo: IBOVESPA...
  Previsões para IBOVESPA geradas com sucesso. (4 registros)
Gerando previsões para o ativo: NASDAQ...
  Previsões para NASDAQ geradas com sucesso. (3 registros)

Geração de previsões concluída para todos os ativos.


In [19]:
import plotly.graph_objects as go

print("Gerando gráficos de comparação de preços reais vs. previstos...")

for asset, data in predictions_df.items():
    print(f"Criando gráfico para o ativo: {asset}")


    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=data['dates'],
        y=data['actual'],
        mode='lines',
        name=f'{asset} - Preço Real',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=data['dates'],
        y=data['predicted'],
        mode='lines',
        name=f'{asset} - Preço Previsto',
        line=dict(color='red', dash='dash')
    ))


    fig.update_layout(
        title=f'Preços Reais vs. Previsões para {asset}',
        xaxis_title='Data',
        yaxis_title='Preço de Fechamento',
        template='plotly_white',
        height=500
    )

    fig.show()

print("Todos os gráficos de comparação foram gerados com sucesso.")

Gerando gráficos de comparação de preços reais vs. previstos...
Criando gráfico para o ativo: BITCOIN


Criando gráfico para o ativo: IBOVESPA


Criando gráfico para o ativo: NASDAQ


Todos os gráficos de comparação foram gerados com sucesso.


## 7. Preparação do Modelo de Machine Learning

### Objetivo
Preparar os dados coletados e transformados para serem utilizados no treinamento de modelos preditivos.

### Etapas Principais
1.  **Seleção de Features e Target**: Foram selecionadas as features ('Open', 'High', 'Low', 'Volume', 'Daily_Return', 'Volatility_7d', 'MA_7') e o preço de fechamento ('Close') como target para cada ativo.
2.  **Normalização dos Dados**: Utilização do `MinMaxScaler` para escalar tanto as features quanto a variável target, garantindo que todos os dados estejam em uma faixa comparável (0 a 1) e evitando que features com valores maiores dominem o treinamento.
3.  **Divisão em Conjuntos de Treino e Teste**: Cada conjunto de dados foi dividido em 80% para treinamento e 20% para teste, permitindo avaliar a capacidade de generalização do modelo.

### Resultado
Dados escalados e prontos para alimentar o modelo de Machine Learning, garantindo um treinamento estável e avaliações justas da performance.

## 8. Treinamento do Modelo de Machine Learning

### Modelo Utilizado
Para a primeira iteração do projeto, optou-se pela **Regressão Linear** (`LinearRegression` da biblioteca `scikit-learn`). Este modelo foi escolhido por sua simplicidade e interpretabilidade, servindo como uma linha de base eficaz para avaliar a complexidade dos dados.

### Processo de Treinamento
1.  **Instanciação do Modelo**: Uma instância do modelo `LinearRegression` foi criada para cada ativo (Bitcoin, Ibovespa, NASDAQ).
2.  **Ajuste (Fit) do Modelo**: O modelo foi treinado utilizando os conjuntos de dados de treinamento (`X_train` e `y_train`) de cada ativo. Durante este processo, o modelo aprende os coeficientes que minimizam o erro entre os valores previstos e os valores reais.

### Resultado
Modelos de Regressão Linear treinados e prontos para realizar previsões sobre os preços de fechamento dos ativos, com base nas features fornecidas.

## 9. Avaliação e Resultados do Modelo de Machine Learning

### Métricas de Avaliação
Para avaliar a performance de cada modelo de Regressão Linear, foram utilizadas as seguintes métricas:
*   **Erro Quadrático Médio (MSE)**: Mede a média dos quadrados dos erros.
*   **Raiz do Erro Quadrático Médio (RMSE)**: A raiz quadrada do MSE, fornecendo o erro na mesma unidade da variável target.
*   **R-squared (R²)**: Indica a proporção da variância na variável dependente que é previsível a partir das variáveis independentes (quanto mais próximo de 1, melhor).

### Resultados Obtidos

*   **IBOVESPA**:
    *   **R-squared**: 0.9995 (Excelente)
    *   **MSE**: 665.59, **RMSE**: 25.79
    *   O modelo demonstrou uma capacidade preditiva excepcional, explicando quase toda a variância nos preços de fechamento.

*   **BITCOIN**:
    *   **R-squared**: 0.9299 (Muito Bom)
    *   **MSE**: 138798.56, **RMSE**: 372.56
    *   Uma boa performance, indicando que a regressão linear capturou a maioria dos movimentos de preço.

*   **NASDAQ**:
    *   **R-squared**: -1.7250 (Muito Ruim)
    *   **MSE**: 52753.04, **RMSE**: 229.68
    *   Um R-squared negativo sugere que o modelo é inadequado para prever os preços da NASDAQ com as features e modelo atuais, performing pior do que uma previsão baseada na média.

### Visualização das Previsões
Gráficos de comparação entre os preços reais e previstos foram gerados para cada ativo, confirmando visualmente a alta precisão para IBOVESPA e BITCOIN, e a falha do modelo para NASDAQ.

### Conclusões e Próximos Passos
*   A Regressão Linear é eficaz para Ibovespa e Bitcoin com as features selecionadas.
*   Para a NASDAQ, é crucial explorar modelos mais complexos (e.g., LSTMs) e features adicionais.
*   Futuramente, buscar aprimorar os modelos com otimização de hiperparâmetros e estratégias de validação mais robustas.



# Análise de Tendências e Previsão de Preços em Mercados Financeiros

## 1. Introdução

Este projeto explora o fascinante e complexo mundo dos mercados financeiros, com foco na análise de tendências e na previsão de preços de ativos-chave. Em um cenário global cada vez mais volátil e interconectado, a capacidade de compreender os movimentos do mercado e antecipar futuras flutuações de preços torna-se um diferencial estratégico. Utilizando dados históricos de índices de mercado como o NASDAQ e o IBOVESPA, bem como da criptomoeda Bitcoin, este trabalho visa demonstrar a aplicação de técnicas de engenharia de dados, análise exploratória e Machine Learning para extrair *insights* valiosos e construir modelos preditivos.

O desafio central reside na inerente imprevisibilidade dos mercados financeiros, onde múltiplos fatores econômicos, políticos e sociais interagem de maneiras complexas. A abordagem proposta busca mitigar essa complexidade através de um pipeline de dados robusto, que abrange desde a ingestão e tratamento de dados até a construção e avaliação de modelos preditivos, culminando em visualizações interativas para facilitar a interpretação dos resultados.

## Gerar Conteúdo do README - Introdução

### Subtask:
Elaborar a seção de Introdução do README, apresentando o tema e o problema abordado pelo projeto.


# Análise de Tendências e Previsão de Preços em Mercados Financeiros

## 1. Introdução

Este projeto explora o fascinante e complexo mundo dos mercados financeiros, com foco na análise de tendências e na previsão de preços de ativos-chave. Em um cenário global cada vez mais volátil e interconectado, a capacidade de compreender os movimentos do mercado e antecipar futuras flutuações de preços torna-se um diferencial estratégico. Utilizando dados históricos de índices de mercado como o NASDAQ e o IBOVESPA, bem como da criptomoeda Bitcoin, este trabalho visa demonstrar a aplicação de técnicas de engenharia de dados, análise exploratória e Machine Learning para extrair *insights* valiosos e construir modelos preditivos.

O desafio central reside na inerente imprevisibilidade dos mercados financeiros, onde múltiplos fatores econômicos, políticos e sociais interagem de maneiras complexas. A abordagem proposta busca mitigar essa complexidade através de um pipeline de dados robusto, que abrange desde a ingestão e tratamento de dados até a construção e avaliação de modelos preditivos, culminando em visualizações interativas para facilitar a interpretação dos resultados.

## 2. Motivação

No cenário atual dos mercados financeiros, caracterizado por alta volatilidade e grande volume de dados, a capacidade de analisar informações de forma eficaz e gerar previsões precisas tornou-se um diferencial competitivo crucial. Investidores e analistas buscam constantemente métodos para entender as tendências, mitigar riscos e otimizar suas estratégias de investimento.

Este projeto surge da necessidade de:

*   **Compreender a Dinâmica do Mercado**: Analisar o comportamento de ativos financeiros chave como NASDAQ, IBOVESPA e Bitcoin, que representam diferentes classes e mercados, oferecendo uma visão abrangente do panorama financeiro global.
*   **Extrair Insights Acionáveis**: Ir além da simples observação de preços, calculando métricas financeiras importantes como retornos diários, volatilidade e médias móveis, que são fundamentais para a tomada de decisão.
*   **Desenvolver Capacidade Preditiva**: Aplicar técnicas de Machine Learning para construir modelos capazes de prever preços de fechamento, auxiliando na identificação de oportunidades e na gestão de riscos.
*   **Visualizar Dados Complexos**: Apresentar os resultados de análises e previsões de forma clara e intuitiva através de dashboards interativos, tornando informações complexas acessíveis e compreensíveis.
*   **Construir um Pipeline Robusto**: Demonstrar a construção de um pipeline completo de Big Data, desde a ingestão e transformação até o armazenamento e visualização de dados financeiros, utilizando ferramentas e bibliotecas modernas.

Através deste projeto, almeja-se não apenas aprimorar a compreensão sobre os mercados financeiros, mas também construir uma solução escalável e replicável que possa ser adaptada para a análise de outros ativos e para a implementação de estratégias de investimento mais sofisticadas, garantindo uma vantagem estratégica em um ambiente de mercado cada vez mais desafiador.

### 3. Objetivo do Projeto

O principal objetivo deste projeto é desenvolver uma solução completa e robusta para análise do mercado financeiro, abrangendo desde a ingestão e tratamento de dados até a aplicação de técnicas avançadas de Machine Learning e visualização interativa. A solução visa:

*   **Construção de um Pipeline de Dados Robusto**: Criar um fluxo de dados eficiente e automatizado para coletar, processar e armazenar informações financeiras de diversas fontes (e.g., Yahoo Finance).
*   **Aplicação de Data Science e Machine Learning**: Utilizar algoritmos de Machine Learning (como Regressão Linear, e futuramente outros modelos mais avançados) para prever preços de ativos financeiros, como Bitcoin, Ibovespa e Nasdaq, e identificar padrões que possam auxiliar na tomada de decisões.
*   **Geração de Insights Acionáveis**: Transformar dados brutos em informações valiosas e apresentá-las através de dashboards interativos e análises detalhadas, permitindo uma compreensão clara do comportamento do mercado e das previsões do modelo.
*   **Avaliação e Melhoria Contínua**: Estabelecer um processo para avaliar a performance dos modelos preditivos e iterar sobre o desenvolvimento para otimizar a precisão e a utilidade da solução.

```markdown
## 4. Metodologia (Pipeline de Dados)

Nossa solução implementa um pipeline de dados robusto e modular para coletar, processar, armazenar e analisar dados financeiros de múltiplos ativos. O fluxo de trabalho é projetado para garantir a integridade dos dados, permitir análises aprofundadas e suportar o desenvolvimento de modelos preditivos, culminando em visualizações interativas para insights rápidos.

### 4.1. Arquitetura do Pipeline
A arquitetura do pipeline é organizada em classes Python, cada uma com uma responsabilidade bem definida, promovendo modularidade e manutenibilidade:

*   **`FinancialDataIngestion`**: Responsável pela coleta de dados de diversas fontes externas.
*   **`FinancialDataTransformation`**: Realiza o processamento, limpeza e enriquecimento dos dados brutos.
*   **`FinancialDataStorage`**: Gerencia o armazenamento persistente dos dados em diferentes formatos.
*   **`FinancialDashboard`**: Cria visualizações interativas para explorar os dados e resultados dos modelos.

### 4.2. Etapas do Pipeline

a. **Fontes de Dados**
Os dados financeiros são coletados de fontes públicas e amplamente reconhecidas. Especificamente, utilizamos o **Yahoo Finance** para obter informações históricas de preços para os seguintes ativos:
    *   **NASDAQ (símbolo: ^IXIC)**: Representando o mercado de tecnologia dos EUA.
    *   **IBOVESPA (símbolo: ^BVSP)**: O principal índice da bolsa de valores brasileira.
    *   **Bitcoin (símbolo: BTC-USD)**: A principal criptomoeda do mercado.

b. **Ingestão de Dados**
A classe `FinancialDataIngestion` é responsável por esta etapa. Ela utiliza a biblioteca `yfinance` para realizar a coleta programática dos dados históricos de preços (Open, High, Low, Close, Volume) para os ativos especificados. A ingestão é parametrizável, permitindo definir o período de coleta dos dados.

c. **Transformação de Dados**
A classe `FinancialDataTransformation` orquestra o processamento dos dados coletados. As principais operações realizadas incluem:
    *   **Limpeza de Dados**: Conversão de tipos de dados (e.g., 'Date' para datetime) e remoção de registros com valores ausentes (`NaN`).
    *   **Cálculo de Retornos**: Geração de métricas financeiras essenciais, como retornos diários (`Daily_Return`) e retornos acumulados (`Cumulative_Return`).
    *   **Indicadores Técnicos**: Cálculo de volatilidade móvel de 7 dias (`Volatility_7d`) e média móvel de 7 dias (`MA_7`) para auxiliar na análise e como *features* para modelos de Machine Learning. Todas as transformações são realizadas utilizando as capacidades de manipulação de dados do `pandas`.

d. **Armazenamento de Dados**
A classe `FinancialDataStorage` gerencia a persistência dos dados. Após a ingestão e transformação, os dados são armazenados de forma estruturada:
    *   Os dados brutos e processados são salvos em formato **CSV** para fácil inspeção e interoperabilidade.
    *   Mais importante, os dados processados são armazenados em formato **Parquet**. Este formato colunar é otimizado para acesso e consulta eficientes, agindo como nosso **'Data Lake'**, facilitando o carregamento rápido para análises subsequentes e treinamento de modelos. O `pandas` é utilizado para operações de leitura e gravação de arquivos.

e. **Consumo de Dados (ML e Visualização)**
Os dados processados e armazenados servem como base para as etapas finais do pipeline:
    *   **Machine Learning**: Os dados são consumidos por modelos de Machine Learning (por exemplo, Regressão Linear) treinados para prever os preços de fechamento futuros dos ativos. Os dados são pré-processados (normalização) e divididos em conjuntos de treino e teste antes de alimentar o modelo.
    *   **Visualização**: A classe `FinancialDashboard` consome os dados processados para gerar uma série de gráficos e tabelas interativas, permitindo a visualização de tendências de preços, retornos, volatilidade e o desempenho das previsões do modelo.

### 4.3. Tecnologias Utilizadas
O pipeline é construído utilizando um conjunto de bibliotecas e ferramentas amplamente empregadas no ecossistema de Data Science em Python:

*   **Python**: Linguagem de programação principal.
*   **pandas**: Para manipulação, limpeza e transformação de dados.
*   **yfinance**: Para ingestão de dados financeiros do Yahoo Finance.
*   **scikit-learn**: Para a implementação e avaliação de modelos de Machine Learning (e.g., `LinearRegression`, `MinMaxScaler`).
*   **Plotly**: Para a criação de visualizações de dados interativas.
*   **Numpy**: Para operações numéricas de alto desempenho.
*   **python-dotenv**: Para gerenciamento de variáveis de ambiente (não explicitamente usado neste trecho, mas comum em projetos de ML/Dados).
```


## 5. Resultados e Visualizações

Esta seção apresenta um resumo dos principais dashboards, gráficos e insights obtidos através da análise dos dados financeiros, bem como a avaliação do modelo de Machine Learning desenvolvido.

### Dashboards e Gráficos

Geramos uma série de visualizações interativas para compreender o comportamento dos ativos (BITCOIN, IBOVESPA, NASDAQ) ao longo do tempo:

*   **Comparação de Preços dos Ativos**: Gráficos de linha que mostram a evolução dos preços de fechamento para cada ativo, permitindo uma análise comparativa direta.
*   **Comparação de Retornos Acumulados**: Visualização do desempenho percentual acumulado de cada ativo, ideal para entender qual ativo gerou maior retorno no período.
*   **Comparação de Volatilidade (7 dias)**: Gráficos de linha que ilustram a volatilidade diária (calculada como desvio padrão dos retornos diários em uma janela de 7 dias), ajudando a identificar períodos de maior risco.
*   **Comparação de Volume de Negociação**: Gráficos de barras que exibem o volume de transações para cada ativo, indicando a liquidez e o interesse do mercado.
*   **Comparação de Médias Móveis (7 dias)**: Gráficos de linha que suavizam as flutuações de preço, mostrando a tendência de curto prazo de cada ativo.
*   **Gráfico de Candlestick**: Visualização detalhada para um ativo específico (ex: BITCOIN), apresentando os preços de abertura, fechamento, máximas e mínimas, essencial para a análise técnica.

### Tabela Resumo dos Ativos

Foi gerada uma tabela de resumo contendo métricas chave para cada ativo, incluindo período analisado, preço inicial e final, retorno total, retorno médio diário, volatilidade e volume médio. Essa tabela oferece um panorama rápido do desempenho e das características de cada ativo.

### Resultados do Modelo de Machine Learning

Implementamos um modelo de Regressão Linear para prever os preços de fechamento dos ativos, e sua performance foi avaliada com as seguintes métricas nos dados de teste:

*   **BITCOIN**:
    *   **MSE**: 52788.5502
    *   **RMSE**: 229.7576
    *   **R-squared**: 0.9743
    O modelo apresentou uma performance muito boa para o Bitcoin, com um R-squared elevado, indicando que a regressão linear conseguiu explicar a maior parte da variância nos preços de fechamento.

*   **IBOVESPA**:
    *   **MSE**: 665.5898
    *   **RMSE**: 25.7990
    *   **R-squared**: 0.9995
    Para o Ibovespa, o modelo demonstrou uma performance excepcional, com um R-squared próximo de 1, o que sugere uma capacidade de previsão quase perfeita com os dados e características atuais.

*   **NASDAQ**:
    *   **MSE**: 52753.0365
    *   **RMSE**: 229.6803
    *   **R-squared**: -1.7250
    O modelo de Regressão Linear performou muito mal para o NASDAQ. Um R-squared negativo indica que o modelo é pior do que simplesmente prever a média dos valores reais, sugerindo que as features atuais ou o tipo de modelo não são adequados para a previsão deste ativo.

### Insights Finais

As visualizações e a avaliação do modelo de Machine Learning revelaram insights importantes:

*   **Boa Capacidade Preditiva para BITCOIN e IBOVESPA**: O modelo de Regressão Linear, utilizando as características selecionadas, mostrou-se eficaz na previsão dos preços de fechamento para BITCOIN e, especialmente, para o IBOVESPA, com altos valores de R-squared.
*   **Necessidade de Investigação para NASDAQ**: A performance insatisfatória para o NASDAQ aponta para a necessidade de explorar modelos mais complexos (como LSTMs ou modelos de séries temporais dedicados), engenharia de features adicionais, ou diferentes abordagens de pré-processamento de dados para este ativo específico.
*   **Potencial para Modelos Avançados**: Embora a Regressão Linear tenha fornecido bons resultados para alguns ativos, a complexidade dos mercados financeiros sugere que modelos mais avançados, como Redes Neurais Recorrentes (LSTMs), poderiam capturar padrões temporais mais intrincados e potencialmente melhorar as previsões para todos os ativos, especialmente para aqueles com maior volatilidade e menor linearidade.


## 6. Conclusões

Neste projeto, desenvolvemos um pipeline completo para ingestão, transformação, armazenamento e análise de dados financeiros, culminando na construção e avaliação de modelos de Machine Learning para previsão de preços de fechamento. Os resultados demonstraram o potencial e os desafios da previsão em mercados financeiros.

### Análise Crítica dos Resultados

Os modelos de Regressão Linear apresentaram performances variadas entre os ativos:

*   **IBOVESPA**: O modelo obteve um desempenho excepcional, com um R-squared de 0.9995. Isso indica que ele foi capaz de explicar quase toda a variância nos preços de fechamento do Ibovespa durante o período de teste, sugerindo uma forte correlação entre as features selecionadas e o preço do ativo.
*   **BITCOIN**: Para o Bitcoin, o modelo também mostrou um bom desempenho, com um R-squared de 0.9743. Embora ligeiramente inferior ao Ibovespa, este valor ainda aponta para uma alta capacidade preditiva, refletindo a eficácia das features em capturar os movimentos de preço desta criptomoeda.
*   **NASDAQ**: Em contraste, o modelo para a NASDAQ falhou significativamente, resultando em um R-squared negativo de -1.7250. Um R-squared negativo indica que o modelo é pior do que simplesmente prever a média dos preços reais, sugerindo que a Regressão Linear com as features atuais não é adequada para capturar a complexidade e a volatilidade do mercado NASDAQ.

Visualmente, os gráficos de comparação confirmaram o alinhamento das previsões com os preços reais para Ibovespa e Bitcoin, e a grande discrepância para a NASDAQ.

### Dificuldades Encontradas

Durante o desenvolvimento do projeto, algumas dificuldades foram notadas:

*   **Coleta de Dados**: A dependência de APIs externas (como Yahoo Finance) pode gerar inconsistências ou lacunas nos dados, especialmente para períodos muito específicos ou ativos menos líquidos. A padronização dos formatos de dados entre diferentes fontes também exigiu etapas de limpeza cuidadosas.
*   **Volatilidade dos Mercados**: A alta volatilidade intrínseca dos mercados financeiros, como observado no Bitcoin e, por vezes, na NASDAQ, torna a previsão de preços uma tarefa desafiadora. Pequenas variações podem ter grande impacto nas métricas de erro.
*   **Seleção de Features**: A escolha das features ('Open', 'High', 'Low', 'Volume', 'Daily_Return', 'Volatility_7d', 'MA_7') foi baseada em indicadores comuns, mas nem sempre se mostrou ideal para todos os ativos, como evidenciado pelo desempenho do NASDAQ.

### Sugestões para Trabalhos Futuros

Para aprimorar ainda mais este projeto, as seguintes abordagens podem ser exploradas:

*   **Modelos Mais Avançados**: Investigar a aplicação de modelos de séries temporais mais sofisticados, como Redes Neurais Recorrentes (RNNs), LSTMs (Long Short-Term Memory) ou Transformers, que são mais adequados para capturar dependências temporais complexas em dados financeiros.
*   **Inclusão de Mais Features**: Expandir o conjunto de features, incluindo:
    *   **Indicadores Técnicos Adicionais**: RSI, MACD, Bandas de Bollinger, etc.
    *   **Dados Fundamentais**: Notícias macroeconômicas, relatórios de empresas, taxas de juros.
    *   **Análise de Sentimentos**: Integrar dados de mídias sociais ou notícias financeiras para capturar o sentimento do mercado.
*   **Otimização de Hiperparâmetros**: Realizar uma otimização sistemática dos hiperparâmetros dos modelos utilizados para encontrar a melhor configuração para cada ativo.
*   **Modelos Ensemble**: Combinar as previsões de múltiplos modelos para mitigar o risco de um único modelo e potencialmente melhorar a robustez e a precisão das previsões.
*   **Estratégias de Validação**: Empregar estratégias de validação cruzada específicas para séries temporais (e.g., walk-forward validation) para garantir uma avaliação mais realista do desempenho do modelo em condições de mercado dinâmicas.

Este projeto serve como uma base sólida para futuras explorações no campo da previsão de mercados financeiros, destacando a necessidade de abordagens adaptáveis e modelos robustos para lidar com a complexidade inerente a esses dados.